# 基础数据应用部分

In [1]:
import os
from moviepy.editor import *
import time
import re
import pandas as pd
import json

# 字符串时、分、秒转换成秒函数
def str2sec(x):
    h, m, s = x.strip().split(':')
    return int(h)*3600 + int(m)*60 + int(s) 

# 字符串时、分、秒、毫秒转换成秒函数
def str2sec_(x):
    h, m, s = x.strip().split(':')
    s,f = s.split(",")
    return str(int(h)*3600 + int(m)*60 + int(s)) + "." + str(f) 


# 将影片放到 source_video 文件夹下
video = os.listdir("source_video/")[0]

# 必须手动的部分

## 剪切掉原片的片头部分

In [4]:
test_start_time = 11  # 这里根据影片开头的时间设置
test_end_time = 20 # 一般20秒之前就开始正式解说了

# 手动测试片头从对应时间切分
video_base =  VideoFileClip("source_video/" + video)
video_base = video_base.subclip(test_start_time,test_end_time)
video_base.write_videofile("temp_data/cut_start.mp4")

Moviepy - Building video temp_data/cut_start.mp4.
MoviePy - Writing audio in cut_startTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video temp_data/cut_start.mp4



Moviepy - Done !
Moviepy - video ready temp_data/cut_start.mp4


## 设置画面截取的范围

In [5]:
# 使用系统自带画图功能选择截取的坐标点
x1,y1 = 0,130 # 裁剪区域左上角坐标
x2,y2 = 1920,960 # 裁剪区域右下角坐标
width,height = None,None # 宽度和高度
x_center,y_center = None,None # X、Y轴中心点坐标

# 剔除上下黑边选取影响范围
video_base =  VideoFileClip("temp_data/cut_start.mp4")
video_base = (video_base.fx(vfx.crop,x1,y1,x2,y2)).resize((1920, 1080))
video_base.write_videofile("temp_data/cut_frames.mp4")

Moviepy - Building video temp_data/cut_frames.mp4.
MoviePy - Writing audio in cut_framesTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video temp_data/cut_frames.mp4



Moviepy - Done !
Moviepy - video ready temp_data/cut_frames.mp4


## 裁剪整个影片

In [6]:
# 计算开始和截至时间
start_time = test_start_time  # 这里根据影片开头的时间设置

video_base =  VideoFileClip("source_video/" + video) # 手动测试片头从对应时间切分
end_time = video_base.duration

# 手动测试片头从对应时间切分
video_base =  VideoFileClip("source_video/" + video)
# 按照测测试号的时间进行截取
video_base = video_base.subclip(test_start_time,end_time)
# 剔除视频无用部分
video_base = (video_base.fx(vfx.crop,x1,y1,x2,y2)).resize((1920, 1080))
video_base.write_videofile("temp_data/"+ video)

Moviepy - Building video temp_data/2022最新奇幻片《大話西遊：緣起》，吳孟達遺作，可惜沒上院線！.mp4.
MoviePy - Writing audio in 2022最新奇幻片《大話西遊：緣起》，吳孟達遺作，可惜沒上院線！TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video temp_data/2022最新奇幻片《大話西遊：緣起》，吳孟達遺作，可惜沒上院線！.mp4



Moviepy - Done !
Moviepy - video ready temp_data/2022最新奇幻片《大話西遊：緣起》，吳孟達遺作，可惜沒上院線！.mp4


# 字幕、文稿处理方法

## 字幕处理方法

将剪映的影片目录放到当下目录 JianyingPro Drafts 放到脚本同级目录，提取剪映中的文档 **draft_content.json** 文件到 **temp_data** 目录下

- 方法1：登录 https://jy.mzh.ren/ 手动加载json文件导出 保存成文档2份（文档1纯文本不包含时间，文档2文本包含时间节点）
- 方法2：自制的解析代码解析字幕内容，直接执行代码保存成文档2份（文档1纯文本不包含时间，文档2文本包含时间节点）

In [7]:
# 加载draft_content.json一键处理文字生成文档
with open("temp_data/draft_content.json",encoding="utf8") as f:
    content_json = f.read()
    content_json = json.loads(content_json)
    
# 将字幕的时间断
content_time_list = content_json["tracks"][1]
# 将字幕的字符串保存成列表
content_list = content_json["materials"]["texts"]

# 构建字幕id 对应时间的 dict
content_time_dict ={}
for i in content_time_list["segments"]:
    id_ = i["material_id"]
    duration = i["target_timerange"]
    content_time_dict[id_] = duration
    
# 时间数字转换时分秒毫秒
def change_time(data):
    milliseconds, milliseconds_ = divmod(data, 1000)
    seconds, milliseconds = divmod(milliseconds, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)

    hours = "0"+ str(hours) if len(str(hours)) != 2 else str(hours)
    minutes = "0"+ str(minutes) if len(str(minutes)) != 2 else str(minutes)
    seconds = "0"+ str(seconds) if len(str(seconds)) != 2 else str(seconds)

    return "{}:{}:{},{}".format(hours, minutes, seconds,milliseconds)

strs_1 = ""
for n in range(len(content_list)):
    content = content_list[n]["content"]
    time_ = content_time_dict[content_list[n]["id"]]
    time_strat = change_time(time_["start"])
    time_end = change_time(time_["start"] + time_["duration"])
    strs_1 = strs_1 + "{}\n".format(content)
# 拼接带字符串的文稿
with open("temp_data/文稿1.txt","w",encoding='utf-8') as f:
    f.write(strs_1)
f.close()    
    
    
strs_2 = ""
for n in range(len(content_list)):
    content = content_list[n]["content"]
    time_ = content_time_dict[content_list[n]["id"]]
    time_strat = change_time(time_["start"])
    time_end = change_time(time_["start"] + time_["duration"])
    strs_2 = strs_2 + "{} --> {}\n{}\n".format(time_strat,time_end,content)

# 拼接带时间、字符串的文稿
with open("temp_data/文稿2.txt","w",encoding='utf-8') as f:
    f.write(strs_2)
f.close()

## 文稿匹配拆分方案2，拆分时间进行进行文稿重塑

In [8]:
# 这里按照方法1处理
with open("temp_data/文稿2.txt",encoding='utf-8') as f:
    txt_list = f.readlines()
    
    
# 判断是否是时间的列表元素 定位符号 -->
data_list = []
for i in range(len(txt_list)):
    # 字符元素判断
    if "-->" in txt_list[i]:
        # 当前索引为 i
        time_str = txt_list[i] # 时间字符串
        # 处理时间字符"00:00:00,133 --> 00:00:01,866\n"
        time_str_1 = time_str.replace("\n","").split(" --> ")[0]
        time_str_2 =time_str.replace("\n","").split(" --> ")[1]
        content = txt_list[i+1].replace("\n","") # 文字字符串
        data_list.append((
            str2sec_(time_str_1),
            str2sec_(time_str_2),
            content
        ))

# 导出全部文稿进行处理      
# 生成文件是 字幕开始时间、结束时间、对应文稿 
# 理论上匹配的就是文稿对应的影片片段
str_df = pd.DataFrame(data_list,columns=["time_str_1","time_str_2","content"])
str_df["new_content"] = ""
str_df.to_excel("temp_data/文稿2.xlsx")
str_df.head(6)

,time_str_1,time_str_2,content,new_content
0,0.100,2.200,致敬了很多原作中的剧情,
1,2.200,4.266,真的要大喊一句爷亲回,
2,4.400,5.300,好了话不多说,
3,5.300,6.433,我们来看剧情,
4,6.533,9.766,至尊宝出生在一个叫花果村的小村庄,
5,9.800,11.933,父亲达叔是一个卖面条的,


## 手动修改文稿2.xlsx 

- 替换新的解说内容到 new_content 
- 删除自己不需要的片段对白，在对应单元格中清空内容

## 修改好的文稿 2.xlsx 放置 temp_mp3 目录下

# 影片素材合成

分几个不同的版本合成，发现有时候字幕和画面对不上的情况会进行微调

1. 百度API接口版本（目前已完成）
2. 科大讯飞API接口版本（还没测试）
3. 阿里API接口版本（还没测试）

# 百度AI接口合成版本

百度AI这个版本是按照次数收费的因此200是个上限，计算每个段落生成的文字合成语音的内容

## API 基础配置设置

In [2]:
from aip import AipSpeech
from base_data.API_Info import *

# 加载API使用的 APP_ID, API_KEY, SECRET_KEY
APP_ID = baidu_api_dict["APP_ID"]
API_KEY = baidu_api_dict["API_KEY"]
SECRET_KEY = baidu_api_dict["SECRET_KEY"]

client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)

# API语言合成参数设置
vol_ = 15 # 音量
per_ = 5118 # 精品发音人选择：度逍遥（精品）=5003，度小鹿=5118，度博文=106，度小童=110，度小萌=111，度米朵=103，度小娇=5
spd_ = 7 # 速度

## 字幕文稿处理

In [3]:
# 文稿处理完毕后读取进行稿件拼接
df = pd.read_excel("temp_mp3/文稿2.xlsx")
# 删除空白的片段内容
df.dropna(axis=0,inplace=True) 
df.reset_index(drop=True,inplace=True)
strs = ""
for i in range(len(df)):
    strs = strs + df["new_content"][i] + "\n"
with open("temp_mp3/文稿2_new.txt",'w',encoding='utf-8') as f:
    f.write(strs)
    
# 使用新的数据重复方案1的步骤生成新的稿子
# 文稿2_new == 文稿1
with open("temp_mp3/文稿2_new.txt",encoding='utf-8') as f:
    txt_data = f.read()
# 将文稿2.xlsx数据格式转换成文稿2.txt按行读取
data_list=df[["time_str_1","time_str_2","new_content"]].values.tolist()

## 字幕文稿按照API长度进行重置

In [4]:
# 累计求不超过200字的文稿和影片按照批次合成
# 拆分 data_list 文稿接近200字的组成一个列表
# 设定单词转换最大的字符长度 为了方式意外设定少一点
each_max_str = 180
# 设定一个总的集合
each_split_list = []
# 设定一个临时的集合
each_list = [] 
# 设定一个空字符串长度累计用
strs_len = 0

for i in data_list:
    strs_len = strs_len + len(i[2]) # 计算单个元素的文字长度进行累计求和
    each_list.append(i) # 将数据加入到临时的集合列表中
    # 判断是否超过单次合成最大文字数
    if strs_len > each_max_str:
        each_split_list.append(each_list) # 超过 each_max_str 个字符了
        each_list = []  # 重置临时集合
        strs_len = 0 # 重置字符累计求和长度
        # 开始重新累计
        strs_len = strs_len + len(i[2]) # 计算单个元素的文字长度进行累计求和
        each_list.append(i) # 将数据加入到临时的集合列表中
        
each_split_list.append(each_list) # 将最后剩余的字符填充到列表中
        
print("一共拆分段落 {} 段".format(len(each_split_list)))

一共拆分段落 17 段


## 分段落文字合成语音mp3

In [7]:
audio_num = 1

for list_each in each_split_list:
    str_each = "\n".join([str_[2] for str_ in list_each])
    result_mp3  = client.synthesis(str_each, 'zh', 1, { 'vol': vol_,'per': per_,'spd':spd_})
    # 识别正确返回语音二进制 错误则返回dict 参照下面错误码
    # 将对应mp3保存到生成文本的目录文件夹下
    if not isinstance(result_mp3, dict):
        with open("temp_mp3/"+str(audio_num)+"_音频素材.mp3", 'wb') as f:
            f.write(result_mp3)
            
    audio_num = audio_num + 1

## 按合成语音的段落点重新剪辑视频

In [8]:
# 提取每个段落要处理的视频的的切分点
video_split_second = []
for i in each_split_list:
    # 起始点第一个元素的0索引
    start_time = i[0][0]
    # 终止点最后一个元素的1索引
    end_time = i[-1][1]
    video_split_second.append((start_time,end_time))
    
# 提取文稿2.xlsx中被删减掉的部分
df_drop = pd.read_excel("temp_mp3/文稿2.xlsx")
drop_list = df_drop.loc[df_drop['new_content'].isnull()].values.tolist()


# 删除连续的重复元素合并时间
index = 0
while index < len(drop_list):
    if drop_list[index][1] == drop_list[index-1][2]:
        # 替换当前indxe的值
        drop_list[index][1] = drop_list[index-1][1]
#         print(drop_list[index])
        # 删除前面的 index - 1的元素
        drop_list.pop(index-1)
        index -= 1
    else:
        pass
#         print(drop_list[index])
    index += 1
    
# 加载处理的原片电影
video_base =  VideoFileClip("temp_data/" + video)

# 设置素材序号
num = 1

L = [] # 全部视频集合，验证结果用

# 遍历需要切分片段的起始和终止时间，进行拆分合成
for s,e in video_split_second:
    
#     print("开始切分 {} 至 {} 时间段".format(s,e))
    # 按照字幕长度切分的时间段起始时间和终止时间 s,e
    # 循环判断要切分的内容是否在其中
    for split_time in drop_list:
        temp_time_list = [] # 临时的时间集合列表用于存储切分的时间点
        temp_video_list = [] # 创建临时单片段的合成素材集合
        # 判断内容为nan的时间点在切分的时间段内则重置时间列表
        if s<split_time[1] and e>split_time[2]:
            temp_time_list.append(s)
            temp_time_list.append(e)
            temp_time_list.append(split_time[1])
            temp_time_list.append(split_time[2])
            # 数据去重重新排列顺序
            temp_time_list = list(set(temp_time_list))
            temp_time_list.sort()
            # temp_time_list 是要切分的视频数据时间点 需要重新两两组合构建新的切分点
            cut_time_list =  [temp_time_list[i:i+2] for i in range(0, len(temp_time_list), 2)]
#             print(cut_time_list)
#             print(s,e)
            for time in cut_time_list:
#                 print("包含空白数据的时间切割部分 {} 至 {}".format(time[0],time[1]))
                cut_video = video_base.subclip(time[0],time[1])
#                 print(cut_video.duration)                  
                temp_video_list.append(cut_video)         

#             print(temp_video_list)
            # 截切好的视频片段合并
            cvc = CompositeVideoClip(temp_video_list)
            # 对视频素材静音操作
            cvc = cvc.fx(afx.volumex, 0)
            # 记录MP4的时间
            video_time = cvc.duration
            # 读取MP3配音文件
            audio = AudioFileClip("temp_mp3/"+ str(num) +"_音频素材.mp3")
            # 记录MP3的时间
            audio_time = audio.duration
            # 给视频进行加速 配合音频文件的字幕
            factor = video_time/audio_time # 倍速播放倍数,视频时间/音频时间就是视频加速的时间
            final_duration = None # 倍速播放持续时间 ，可以为空表使全部
            cvc_video = cvc.fx(vfx.speedx, factor , final_duration)
            # 视频和音频合成在一起
            cvc_video = cvc_video.set_audio(audio)
            # 合成无片头文件
#             cvc_video.write_videofile("temp_mp4/"+str(num)+"_视频素材.mp4")
            print(num ,cvc_video.duration)
            L.append(cvc_video) # 临时计算用
            
            break
    else:
        # 如果没有切分的片段直接剪辑成素材
        temp_video = video_base.subclip(s,e)
        # 对视频素材静音操作
        temp_video = temp_video.fx(afx.volumex, 0)
        # 记录MP4的时间
        video_time = temp_video.duration
        # 读取MP3配音文件
        audio = AudioFileClip("temp_mp3/"+ str(num) +"_音频素材.mp3")
        # 记录MP3的时间
        audio_time = audio.duration
        # 给视频进行加速 配合音频文件的字幕
        factor = video_time/audio_time # 倍速播放倍数,视频时间/音频时间就是视频加速的时间
        final_duration = None # 倍速播放持续时间 ，可以为空表使全部
        temp_video = temp_video.fx(vfx.speedx, factor , final_duration)
        # 视频和音频合成在一起
        temp_video = temp_video.set_audio(audio)
        print(num , temp_video.duration)
        L.append(temp_video) # 临时计算用
#         temp_video.write_videofile("temp_mp4/"+str(num)+"_视频素材.mp4")
    # 序号自增
    num = num + 1

1 34.88
2 35.21
3 35.6
4 35.5
5 35.14
6 34.16
7 34.88
8 34.06
9 35.5
10 37.66
11 35.68
12 36.04
13 35.71
14 37.15
15 35.75
16 35.57
17 19.62


## 合并全部素材

In [ ]:
# 文件进行保存到结果文件中
cvc_result_viedo = concatenate_videoclips(L)
cvc_result_viedo.write_videofile("temp_mp4/合成完整素材.mp4")

## 添加水印图

In [2]:
# 加载素材视频
video = VideoFileClip("temp_mp4/合成完整素材.mp4").resize((1920,1080))
start = VideoFileClip("base_data/片头.mp4").resize((1920,1080))

# 视频中添加水印
logo = (
    ImageClip("./base_data/水印1.png")
#         .set_start(start.duration)
        .set_duration(video.duration)  # 水印持续时间
        .resize(height=200)  # 水印的高度，会等比缩放
        .set_pos(("left", "top"))  # 水印的位置
)

# 文件进行保存到结果文件中
cvc = CompositeVideoClip([video,logo])
cvc = concatenate_videoclips([start,cvc])
# 合成无片头文件
cvc.write_videofile("temp_mp4/添加水印的素材.mp4")

Moviepy - Building video temp_mp4/添加水印的素材.mp4.
MoviePy - Writing audio in 添加水印的素材TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video temp_mp4/添加水印的素材.mp4



Moviepy - Done !
Moviepy - video ready temp_mp4/添加水印的素材.mp4


# 算法加工避免机器检测

## 去重机器算法测试部分

In [2]:
from base_data.algorithm_process import *

# 获取MP4路径信息
mp4_data_list = os.listdir("temp_mp4")
mp4_data_list

['合成完整素材.mp4', '添加水印的素材.mp4']

In [3]:
# 获取MP4路径信息
mp4_data_list = os.listdir("temp_mp4")

for name in mp4_data_list:
    if "水印的素材" in name:
        in_path = 'temp_mp4/'+ name
        # 机器加工合成算法
        process_function_1(in_path)

Moviepy - Building video result_video/CNN色彩调整.mp4.
MoviePy - Writing audio in CNN色彩调整TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video result_video/CNN色彩调整.mp4



Moviepy - Done !
Moviepy - video ready result_video/CNN色彩调整.mp4
